In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
import pandas as pd
import re

df = pd.read_csv('./gdrive/MyDrive/data/Tweets.csv')

In [8]:
df.dropna(subset=['text'], inplace=True)
df.head()

,textID,text,selected_text,sentiment,label
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,1
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0
2,088c60f138,my boss is bullying me...,bullying me,negative,0
3,9642c003ef,what interview! leave me alone,leave me alone,negative,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,0


In [9]:
slangs = pd.read_json('./gdrive/MyDrive/data/slangs.csv')
slangs.reset_index(inplace=True)
slangs.columns = ["slang", "meaning"]
slangs.head()

,slang,meaning
0,$,dollar
1,4ao,for adults only
2,IG,instagram
3,a.m,before midday
4,a3,anytime anywhere anyplace


In [26]:
# Text with slang words
def remove_slang(text):
    # Create a dictionary from the DataFrame
    slang_dict = dict(zip(slangs['slang'], slangs['meaning']))

    # Function to replace slang with meanings using word boundaries
    def replace_slang_with_meaning(match):
        slang_word = match.group(0)
        return slang_dict.get(slang_word, slang_word)

    # Use regular expressions with word boundaries to find and replace slang words
    pattern = r'\b(?:' + '|'.join(map(re.escape, slang_dict.keys())) + r')\b'
    result = re.sub(pattern, replace_slang_with_meaning, text)
    return result
original_text = " What`s sad is that I actualy had to google that term. That scks, tho."

slang_removed_text = remove_slang(original_text)
print(slang_removed_text)

from spellchecker import SpellChecker
import string
import re

spell = SpellChecker()
spell.word_frequency.load_words(['microsoft', 'apple', 'google', 'twitter', 'https'])
spell.word_frequency.load_words(list(string.ascii_lowercase))

def correct_spelling(text):
    # Initialize corrected_words to store the corrected text
    corrected_words = []

    # Initialize a variable to keep track of the current word
    current_word = ""
    
    
    for char in text:
        # Check if the character is a letter or an apostrophe (common in contractions)
        if char.isalpha() or char == "'":
            current_word += char
        else:
            if current_word:
                # Check and correct the current word
                corrected_word = spell.correction(current_word)

                # Append the corrected word to the list
                corrected_words.append(corrected_word)

                # Reset the current word
                current_word = ""

            # Append the non-alphabetic character (punctuation, space, etc.) to the list
            corrected_words.append(char)

    # If there's a remaining word at the end of the text, correct and append it
    if current_word:
        corrected_words.append(spell.correction(current_word))

    # Join the corrected words back into a sentence
    corrected_words = [x for x in corrected_words if x is not None]
    corrected_text = "".join(corrected_words)

    return corrected_text


# Example usage
spelling_corrected_text = correct_spelling(slang_removed_text)
print(spelling_corrected_text)

import contractions
def fix_contractions(text):
    return contractions.fix(text)


contractions_corrected_text = fix_contractions(spelling_corrected_text)
contractions_corrected_text

regStr = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"


def pre_process_text(texts):

    processed_texts = []

    for text in texts:
        text = text.lower()
        matches = re.findall(regStr, text)
        matches = {m[0] for m in matches}

        groups = [group for group in re.split(regStr, text) if group]

        cleanedText = ""

        for part in groups:
            if part in matches:
                cleanedText += part + " "
            else:
                cleanedText += correct_spelling(fix_contractions((remove_slang(part)))) + " "
        
        processed_texts.append(cleanedText)
    return processed_texts
    # return [fix_contractions(correct_spelling(remove_slang(text))) for text in texts]


 What`s sad is that I actualy had to google that term. That scks, though.
 What`s sad is that I actually had to google that term. That sucks, though.


In [27]:
sample = df.sample(n = 100, random_state=42)
sample['pre'] = pre_process_text(sample['text'])


In [28]:
sample.iloc[20:40]

,textID,text,selected_text,sentiment,label,pre
20183,41341f5395,Massive morning.. I`m stuffed now,Massive morning.. I`m stuffed now,neutral,1,massive morning.. i`m stuffed now
2831,9c56dcbab3,I wish I had company,I wish I had company,negative,0,i wish i had company
21210,d43a0c0760,i`m on my mobile so it won`t let me but i ca...,t i can`t stop thinking about you,negative,0,i`m on my mobile so it won`t let me but i ca...
12029,94dcf0ec77,Today is check in day for week 12 of the Artis...,time to celebrate an,positive,2,today is check in day for week 12 of the artis...
12812,0fe86fc464,where are you going tonight?!,where are you going tonight?!,neutral,1,where are you going tonight?!
11029,465d430992,I wish I could go. See ya at Warped for sure...,I wish I could go,positive,2,i wish i could go. see ya at warped for sure...
16793,06ca54fe1a,Intel gfx driver situation much better with re...,"Suspend working again, fewer mem leaks",negative,0,intel of driver situation much better with rec...
21660,6353181ff8,"Once again, I`m dealing with cancer in my dire...",Bugger Cancel sucks!,negative,0,"once again, i`m dealing with cancer in my dire..."
24207,da6fe31755,Went to see 'UP'. It was a really good movie t...,Went to see 'UP'. It was a really good movie t...,positive,2,went to see up. it was a really good movie tha...
4891,2d126f6c4f,xD hello that`s an important date!,`s an important date!,positive,2,ad hello that`s an important date!


In [29]:
df['text'] = pre_process_text(df['text'])

In [35]:

df.to_csv('./gdrive/MyDrive/data/tweets_preprocessed.csv', index=False)